# Lesson 02 - CRAM Basics

CRAM stands for Cognitive Robot Abstract Machine. Today we are working with the CRAM plan-executive, which is a framework for writing high-level procedures for robots. High-level, meaning, the procedures in this framework are agnostic of the underlying platform. They can be executed on different robots (PR2, TiaGo, Pepper, HSR, Boxy, Donbot), in different simulations (Bullet, Unreal, MuJoCo), with different motion planners (Giskard, MoveIt, MoveBase, NaiveKinematics), knowledge representations (KnowRob, RALF, internal heuristics), perception systems (Robosherlock, RoboCudo, sim-based perception)... All of these can be easily exchanged by switching out the process-modules for a specific platform, which is like plugging in a different pair of headphones or connect speakers instead, and still hear the same music. The high-level plan stays the same, the music stays the same, but the target output changes. Our platform today is Bullet Physics, a lightweight simulator, that's tightly coupled into CRAMs geometric reasoning.

If you are using the headless Docker image, make sure to run the following launchfile before starting the JupyterNotebook webserver.

`roslaunch cram_pr2_pick_place_demo sandbox.launch`

## Compiling the CRAM code

Steel Bank Common Lisp (SBCL) is the interpreter for our code. To use CRAM, we first need to compile the code to make it available.`asdf:load-system` loads an ASDF (Another System Definition Facility) system into the SBCL kernel. You will remember this line from lesson 0.

In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)

Every time you reboot Docker or restart the kernel, even when you switch to a different lesson, you need to compile the package first. Every lesson has its own SBCL kernel.

When the pick & place package is loaded, it registers lots of initialization functions. The following will boot up the current configuration of initialization. It can also be used to reboot the whole simluation system.

In [ ]:
;; Warning: resets the core processes of the simulation
(roslisp-utilities:startup-ros)

If you don't want to reboot all the CRAM processes with `startup-ros` but only clear the Bullet World of its objects, use this

In [ ]:
;; Warning: resets simulation and deletes objects
(demo::init-projection)

When the simulation hangs up, which happens sometimes, this may bring it back to life. If it doesn't, hit that *Stop* button at the top to kill leftover processes and try again. Eventually you may need to restart the Docker process.

In [ ]:
;; Warning: respawning the window may result in a kernel crash. Restart the lecture via docker to recover from that.
(btr-utils:reset-debug-window)

## The Tutorial Package

The following block defines a new package, called *cram-bullet-world-tutorial*. This is the package we're going to write our code in.

In [ ]:
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)                         ;; an alias for the package
  (:use :common-lisp :cram-prolog :desig :exe)) ;; inherit other packages' namespace to make our code better readable

As you remember from the previous lesson, packages are like namespaces. Our package has a nickname, which we can use to switch into the package.

In [ ]:
(in-package :btw-tut)

There is the function `describe` that gives explanation about symbols and how they are bound. Let's define a global parameter to check this out.

In [ ]:
(defparameter *descriptive-parameter-name* "Lisp rules")

In [ ]:
(describe '*descriptive-parameter-name*)

In [ ]:
(describe '*package*)

## Poses in 3D space

In 3D space we can define a point with x, y and z. We use the `cl-transforms` package to do so.

In [ ]:
(cl-transforms:make-3d-vector 1.0 2.0 3.0)

A point is just a point. Every object is at a certain position in the room, but we don't know how it is oriented. Orientations are commonly represented as a rotation matrix, rotation vector, euler angles or quaternion. CRAM works with quaternions, which are very hard to read for humans, but are very nice to work with. Like rotation matrices, quaternions can the concatenated to calculate a chain of relative orientations.

In [ ]:
(cl-transforms:make-identity-rotation)

See that a quaternion is composed of four values: ax, ay, az, and w. Since quaternions are so hard to construct by hand, I usually translate them from euler angles.

In [ ]:
(cl-transforms:euler->quaternion)                 ;; is the identity rotation
(cl-transforms:euler->quaternion :az pi)          ;; rotates 180 degree around the z-axis. The last value is float-imprecision.
(cl-transforms:euler->quaternion :az (* pi 0.5))  ;; rotates 90 degree to the left
(cl-transforms:euler->quaternion :az (* pi -0.5)) ;; rotates 90 degree to the right

(describe 'cl-transforms:euler->quaternion)

The difference between a position and a pose is, that a position only describes a point, while a pose is the combination of position and orientation.

In [ ]:
(cl-transforms:make-pose (cl-transforms:make-3d-vector 1.0 2.0 3.0)
                         (cl-transforms:make-identity-rotation))

What's missing is a reference to something in the room. Every pose should have a reference frame, something that already has a fixed pose in the world, such that we can define a pose w.r.t. that other pose in the world. In our case this origin frame is called `map`. Such a relative pose also gets a `stamp` (like in time-stamp), but we can ignore that for now.

In [ ]:
(cl-transforms-stamped:make-pose-stamped "map" 0.0
                                         (cl-transforms:make-3d-vector 1.0 2.0 3.0)
                                         (cl-transforms:make-identity-rotation))

To make our code small, let's define a function that constructs a pose for us

In [ ]:
(defun make-pose (frame point euler)
    (declare (type string frame)      ;; verifies the type of 'frame' as string
             (type list point euler)) ;; and list for 'point' and 'euler'
    ;; this is the documentation string
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler.
    frame - a string of the reference frame in the world.
    point - given as a list of x y z coordinates, 
    euler - given as rotations around the x, y and z axis. (right-hand-rule)"
    ;; this checks if the arguments 'point' and 'euler' both have length 3
    (if (and (eq (length point) 3) (eq (length euler) 3))
        ;; destructuring-bind can assign the contents of a list to local variables.
        ;; here it binds the 3 values of the 'euler' list to ax, ay and az
        ;; mapcar with eval is required here to evaluate formulae like (* pi 0.5)
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        ;; else-branch
        (format T "point ~a or euler ~a is not of length 3" point euler)))

In [ ]:
(make-pose "map" '(1.0 2.0 3.0) '(0.0 0.0 (* pi 0.5)))

## Visualizing

All these poses seem pretty abstract, so let's visualize them in Bullet. The function `btr:add-vis-axis-object` can be applied to poses, physical objects and robot parts.

In [ ]:
;; try and mess around with the poses values
(btr:add-vis-axis-object (make-pose "map" '(0.0 0.0 2.0) '((* pi 0.0) (* pi 0.0) (* pi 0.0))))

These are some examples to visualize the pose of parts of the environment and the robot.

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-environment-object) "sink_area_left_middle_drawer_handle"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "l_gripper_tool_frame"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "base_laser_link"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "torso_lift_link"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "wide_stereo_optical_frame"))

To get a full list of the environment and robot parts, this will retreive all possible frames. We use Alexandria here to extract the keys of the hash-table, in which the links of the environment are defined.

In [ ]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-environment-object) 'btr:urdf) 'cl-urdf:links))

In [ ]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-robot-object) 'btr:urdf) 'cl-urdf:links))

We can also visualize areas. The following calculates and shows the distribution of poses that are suitable for the PR2 to stand, in order to reach the sink-area-left-middle-drawer-handle of the kitchen.

In [ ]:
;; this takes a while to compute. lay back and observe.
(btr-utils:visualize-designator-costmaps
 (a location
    (reachable-for pr2)
    (arm left)
    (object (an object
                (type cupboard)
                (urdf-name sink-area-left-middle-drawer-handle)
                (part-of iai-kitchen)))))

## Designators
That previous code block seems wierd. It looks completely different from the Lisp code that we previously saw. That's because it's a designator.

Designators are underspecified, symbolic representations of something that's not evaluated yet. We can describe locations, objects and actions with that. As soon as we need a discrete pose of such a location, we can sample a pose of it and work with that. If that pose doesn't work well, we can take the next poose from that distribution. For our cases though, we can specify a location designator to just be one pose.

### Locations

In [ ]:
(a location)

In [ ]:
(let ((?some-locally-defined-pose (make-pose "map" '(0.0 0.0 2.0) '(0.0 0.0 0.0))))
     (a location
        (pose ?some-locally-defined-pose)))

Note that the local variable has a `?` front of its name. This is common notation for values in a designator. Every time you want to use any kind of value in a designator, it must be defined as a variable starting with `?`. That is, such that the location designator identifies the variable to be evaluated. Evaluating a discrete value for a designator is called `referencing`.

In [ ]:
;; let* can re-use locally defined variables within the definition
(let* ((?some-locally-defined-pose (make-pose "map" '(0.0 0.0 2.0) '(0.0 0.0 0.0)))
       (?some-location-designator (a location
                                     (pose ?some-locally-defined-pose))))
     (reference ?some-location-designator))

Designator resolution is done by evaluating the key-value notation of the designator in logical facts, similar to first-order-logic programming, as it is done in Prolog. This is what makes designators to powerful. Since they are evaluated lazily, there may be more than one solution for the same designator.

Let's dig deeper into that.

### Objects

In [ ]:
(an object)

An object designator, like for locations, can stay underspecified until we assign it to a specific object.

In [ ]:
(an object
    (type bottle))

In [ ]:
;; this won't work
(reference (an object (type bottle)))

Some designators can not be resolved, because there is no solution for them yet. Let's give it a solution. First, let's reset the simulation.

In [ ]:
(demo::initialize)

The robot should stand in it's origin pose. Now spawn a bottle into the world.

In [ ]:
(btr-utils:spawn-object 'that-one-bottle-in-the-front  
                        :bottle 
                        :pose (make-pose "map" '(1.5 0.0 1.5) '(0 0 0)))

And to reference it, we need to perceive it.

### Actions

In [ ]:
(urdf-proj:with-simulated-robot
  (perform (an action
               (type detecting)
               (object (an object
                           (type bottle))))))

The resulting object designator is now resolved, by detecting the object of type bottle in front of the robot. Don't mind that the bottle fell down into the sink. Performing an action also simulates the Bullet World.

`urdf-proj:with-simulated-robot` is a macro that specifies, what process-modules we use for performing an action. Remember in the first paragraph of this lecture, where all plans in CRAM are basically platform-independent. This is that. With that macro we say, that the action should be executed on the simulated robot.

An action in itself is not very spectacular until it's performed. `perform` references an action designator and translates its key-value pairs into parametrization for a Lisp function. If any parameters are missing, they are inferred through massive first-order-logic mechanisms. Every action must have a `type` at least, and what other parameters are required depends on the type of action. 

In [ ]:
(btr-utils:kill-all-objects) ;; cleanup objects

;; place two bottle directly behind each other, while that-one-bottle-in-the-front occludes sight on that-other-bottle-in-the-back
(btr-utils:spawn-object 'that-one-bottle-in-the-front 
                        :bottle 
                        :pose (make-pose "map" '(1.45 0.0 0.96) '(0 0 0)))
(btr-utils:spawn-object 'that-other-bottle-in-the-back
                        :bottle 
                        :pose (make-pose "map" '(1.55 0.0 0.96) '(0 0 0)))

;; execute perceiving any bottle two times, while deleting one in between
(urdf-proj:with-simulated-robot
 (let (first-bottle second-bottle) ;; define two empty local variables
      ;; perceive a bottle
      (setf first-bottle 
            (perform (an action
                         (type detecting)
                         (object (an object
                                     (type bottle))))))
      ;; remove the bottle in the front
      (btr-utils:kill-object 'that-one-bottle-in-the-front)
      ;; perceive again
      (setf second-bottle 
            (perform (an action
                         (type detecting)
                         (object (an object
                                     (type bottle))))))
      (format NIL "first bottle:~%~a~%~%second bottle:~%~a" first-bottle second-bottle)))

See the outprint for the first and the second bottle and check the objects' name. Mess around with the positioning of the two bottles, reset them and execute the actions again. The one in the back will never be perceived first, because the robot can't see it. Only when we remove the one in the front, he can.


This is it for the CRAM basics. To dig deeper, head to lesson 3 about picking and placing objects.